In [1]:
import pandas as pd
import datetime
import numpy as np

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import jupyter_dash
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from IPython.display import display
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [2]:
# pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('data/df2_shifted.csv', encoding='utf-8')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df

,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,concert_event_counts,concert_event_durations_seconds_7_day_rolling_mean,biking_durations_seconds,biking_counts,biking_duration

In [4]:
from pycaret.time_series import *
# metric = "mape"
metric = "R2"

exclude = [
           # "naive", 
           # "snaive", 
            'auto_arima',
            'arima',
          ]
fig_kwargs={'renderer':'browser', 
            # "height":500, 
            # "width":1000
           }

In [5]:
s = setup(df, target='Mood', fold=1, session_id=123,
          ignore_features = ['Mood_shifted'],
          # normalize = True,
          # normalize_method = 'minmax'
          # transformation = True, 
          # transform_target = True, 
          # remove_multicollinearity = True, 
          # multicollinearity_threshold = 0.95
          # fold_strategy='expanding, sliding, rolling'
          fh=7,
         )

,Description,Value
0,session_id,123
1,Target,Mood
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(1165, 315)"
5,Transformed data shape,"(1165, 315)"
6,Transformed train set shape,"(1164, 315)"
7,Transformed test set shape,"(1, 315)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [6]:
add_metric('evs', 'EVS', explained_variance_score)

Name                                                               EVS
Display Name                                                       EVS
Score Function       <function explained_variance_score at 0x7fb439...
Scorer                           make_scorer(explained_variance_score)
Target                                                            pred
Args                                                                {}
Greater is Better                                                 True
Custom                                                            True
Name: evs, dtype: object

In [7]:
add_metric('R2', 'R2', r2_score)

Name                                                    R2
Display Name                                            R2
Score Function       <function r2_score at 0x7fb439193a60>
Scorer                               make_scorer(r2_score)
Target                                                pred
Args                                                    {}
Greater is Better                                     True
Custom                                                True
Name: R2, dtype: object

In [8]:
best = compare_models(sort=metric, 
    turbo=False, 
    exclude=exclude)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,EVS,R2,TT (Sec)
naive,Naive Forecaster,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,nan,4.6200
grand_means,Grand Means Forecaster,0.4218,0.2194,0.1230,0.1230,0.0615,0.0596,1.0000,nan,3.7700
snaive,Seasonal Naive Forecaster,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,nan,3.6100
croston,Croston,0.0003,0.0001,0.0001,0.0001,0.0000,0.0000,1.0000,nan,3.4600
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1796.3437,934.5168,523.6746,523.6746,261.8373,2.0000,1.0000,nan,4.7400
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.0107,0.5258,0.2946,0.2946,0.1473,0.1372,1.0000,nan,5.2100
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,442.7347,230.3251,129.0671,129.0671,64.5336,1.9399,1.0000,nan,3.6500
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.3171,0.1650,0.0924,0.0924,0.0462,0.0452,1.0000,nan,5.2800
lar_cds_dt,Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.1151,0.0599,0.0335,0.0335,0.0168,0.0166,1.0000,nan,4.0900
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.4141,0.2154,0.1207,0.1207,0.0604,0.0586,1.0000,nan,3.2200


In [10]:
predict_model(best)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,EVS,R2
0,Naive Forecaster,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,nan


,y_pred
2023-03-10,2.0


In [16]:
grand_means = create_model('grand_means')

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,EVS,R2
0,2023-03-08,0.4218,0.2194,0.1230,0.1230,0.0615,0.0596,1.0000,nan
Mean,NaT,0.4218,0.2194,0.1230,0.1230,0.0615,0.0596,1.0000,nan
SD,NaT,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,nan


In [18]:
grand_means.plot_model(plot="train_test_split")

AttributeError: 'NaiveForecaster' object has no attribute 'plot_model'

In [19]:
grand_means.plot_model(plot="cv")

AttributeError: 'NaiveForecaster' object has no attribute 'plot_model'

In [14]:
best.plot_model(plot="diff")

AttributeError: 'NaiveForecaster' object has no attribute 'plot_model'

In [15]:
best.plot_model(plot="diagnostics")

AttributeError: 'NaiveForecaster' object has no attribute 'plot_model'